## Import Required Libraries

In [54]:
import numpy as np
import json
import glob
import re

#Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.corpora import Dictionary

#spacy
import spacy
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
from nltk.stem.wordnet import WordNetLemmatizer

#vis
import pyLDAvis
import pyLDAvis.gensim_models
#nltk.download(punkt)
#nltk.download(wordnet) 


### Data Input 

In [55]:
# #Subject to change, only using one file to train initial model. 
# corpus = "So Im a young first gen college graduate, my dad joined the military as soon as he could and my mom dropped out at 16. I never really understood how different I was until I started hanging out with people in the same major as me (Marine Science). I always felt like I missed a crucial stage growing up where my interests wouldve been nurtured. Most of my studies I just did independently. I hated my teachers in high school because they only seemed to provide their helping hands to the kids who needed it least. I grew a bit of an unhealthy resentment to the preppy kids in my angst-ridden phase. I never really thought of going to college, it wasnt talked about, even with my other siblings. I just kind of wound up there and more or less, ended up winging it. Fast forward a few years and I am succeeding as best I can in classes and still feel like Im missing something despite usually being near the top of most of my classes, doing climate change research as an undergraduate, leading study sessions with people in the honors college at my university and having many intelligent and motivating friends winning prestigious scholarships. I always felt like I got the concepts of the classes but everybody else seemed to understand the networking and scholarship and opportunity aspects of college. I graduated from a big school with my degree, my peers thought very highly of me, but I always felt less than everyone else. I felt like everyone else had a plan, knew what they were doing. So fast forward some more and here I am. I finally got my first good job in my field and I never really felt more out of place than here. I want to fit in, everybody is so nice, professional and scholarly. All I know how to do is memorize information and regurgitate it on a test and somehow do well. It just feels weird. Maybe its a confidence thing? I know I can do whatever is needed to be done, I just dont know if I have any thing new to offer or worthwhile to contribute."

In [56]:
def load_data(file):
    with open (file, "r", encoding="utf-8") as f:
        data = json.load(f) 
    return (data)

def write_data(file, data):
    with open (file, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4)
stories = load_data("local_data.json")["File"]
print(stories[0][:100])


My mum had me at 15 years. No idea who my dad is. I grew up with a single mum who would spend every 


### Pre-process and vectorize the documents 

In [57]:
#In case prompted to install stopwords, use "nltk.download(stopwords)".
#Lemmatization
stops = set(stopwords.words("english"))

def lemmatization(texts, allowed_postags=["NOUN", "ADJ", "VERB", "ADV"]):
    #Use "python -m spacy download en_core_web_sm" if error [E050]
    nlp = spacy.load("en_core_web_sm",disable=["parser","ner"])
    texts_out = []
    for text in texts:
        doc = nlp(text)
        new_text = []
        for token in doc:
            if token.pos_ in allowed_postags:
                new_text.append(token.lemma_)
        final = " ".join(new_text)
        texts_out.append(final)
        
    return(texts_out)

lemmatized_texts = lemmatization(stories)
print(lemmatized_texts[0][:100])

C:\Users\MoRevolution\AppData\Roaming\Python\Python310\site-packages\catalogue\__init__.py:135: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
C:\Users\MoRevolution\AppData\Roaming\Python\Python310\site-packages\catalogue\__init__.py:135: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
C:\Users\MoRevolution\AppData\Roaming\Python\Python310\site-packages\catalogue\__init__.py:135: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
C:\Users\MoRevolution\AppData\Roaming\Python\Python310\site-packages\catalogue\__init__.py:135: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entr

C:\Users\MoRevolution\AppData\Roaming\Python\Python310\site-packages\catalogue\__init__.py:123: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
C:\Users\MoRevolution\AppData\Roaming\Python\Python310\site-packages\catalogue\__init__.py:123: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
C:\Users\MoRevolution\AppData\Roaming\Python\Python310\site-packages\catalogue\__init__.py:123: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
C:\Users\MoRevolution\AppData\Roaming\Python\Python310\site-packages\catalogue\__init__.py:123: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entr

mum year idea dad grow single mum spend last dollar coke say poor understatement amount government a


In [58]:
#deaccenting and removing stop words
def gen_words(texts):
    final = []
    for text in texts:
        new = gensim.utils.simple_preprocess(text, deacc=True)
        final.append(new)
    return (final)

data_words = gen_words(lemmatized_texts)
print (data_words[0][0:20])

['mum', 'year', 'idea', 'dad', 'grow', 'single', 'mum', 'spend', 'last', 'dollar', 'coke', 'say', 'poor', 'understatement', 'amount', 'government', 'assistance', 'get', 'mother', 'addict']


In [59]:
#BIGRAMS AND TRIGRAMS
bigram_phrases = gensim.models.Phrases(data_words, min_count=2,threshold=50)
trigram_phrases = gensim.models.Phrases(bigram_phrases[data_words],threshold=50)

bigram = gensim.models.phrases.Phraser(bigram_phrases)
trigram = gensim.models.phrases.Phraser(trigram_phrases)

def make_bigrams(texts):
    return(bigram[doc] for doc in texts)

def make_trigrams(texts): 
    return(trigram[bigram[doc]] for doc in texts)

data_bigrams = make_bigrams(data_words)
data_bigrams_trigrams = make_trigrams(data_bigrams)

data_bigrams_trigrams = list(data_bigrams_trigrams)
print(data_bigrams_trigrams[:2])
   
#texts = [word.encode("utf-8").split(/)[0] for word in texts]
# lemmatize the document(Other methods can be used to execute this. This method is best when working on collab or with gensim V3,gensim.utils has been dropped for V4)
# texts = [[word.decode("utf-8").split(/)[0] for word in lemmatize( .join(line), allowed_tags=re.compile((NN)), min_length=5)] for line in texts]
#trigrams_ = [t for t in trigram[bigram[sent]] if t.count( ) == 2]

[['mum', 'year', 'idea', 'dad', 'grow', 'single', 'mum', 'spend', 'last', 'dollar', 'coke', 'say', 'poor', 'understatement', 'amount', 'government', 'assistance', 'get', 'mother', 'addict', 'move', 'around', 'lot', 'go', 'different', 'school', 'grow', 'have', 'food', 'common', 'occurrence', 'homeless', 'period', 'time', 'kid', 'wash', 'public', 'restroom', 'time', 'time', 'send', 'other', 'relative', 'live', 'sexually', 'abuse', 'multiple', 'occasion', 'keep', 'year', 'kid', 'terrifying', 'speak', 'already', 'live', 'shaky', 'unstable', 'world', 'so', 'uproot', 'last', 'foundation', 'even', 'drug', 'addle', 'mother', 'unthinkable', 'anyway', 'fast', 'forward', 'try', 'really_hard', 'school', 'mean', 'really_hard', 'only', 'way', 'see', 'get', 'hole', 'in', 'mum', 'drop', 'school', 'know', 'never', 'want', 'end', 'get', 'job', 'day', 'th', 'birthday', 'country', 'legal', 'age', 'start', 'work', 'day', 'year', 'now', 'spend', 'day', 'unemployed', 'work', 'save', 'as', 'much', 'mum', 'tel

In [60]:
pip install pattern

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [61]:
#nltk.download(omw-1.4)

In [62]:
#TF-IDF REMOVAL
from gensim.models import TfidfModel

id2word = corpora.Dictionary(data_bigrams_trigrams)

texts = data_bigrams_trigrams

corpus = [id2word.doc2bow(text) for text in texts]
print (corpus[0][0:20])

tfidf = TfidfModel(corpus, id2word=id2word)

low_value = 0.03
words  = []
words_missing_in_tfidf = []
for i in range(0, len(corpus)):
    bow = corpus[i]
    low_value_words = [] #reinitialize to be safe. You can skip this.
    tfidf_ids = [id for id, value in tfidf[bow]]
    bow_ids = [id for id, value in bow]
    low_value_words = [id for id, value in tfidf[bow] if value < low_value]
    drops = low_value_words+words_missing_in_tfidf
    for item in drops:
        words.append(id2word[item])
    words_missing_in_tfidf = [id for id in bow_ids if id not in tfidf_ids] # The words with tf-idf socre 0 will be missing

    new_bow = [b for b in bow if b[0] not in low_value_words and b[0] not in words_missing_in_tfidf]
    corpus[i] = new_bow

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1)]


### Training LDA Model 

In [63]:
from gensim.models import LdaModel

# Set training parameters.
num_topics = 10
chunksize = 2000
passes = 20
iterations = 400
eval_every = 1  # Dont evaluate model perplexity, takes too much time

lda_model = LdaModel(
    corpus=corpus,#model is trained on all stories except for the last one. Used as a test document later on. 
    id2word=id2word,
    chunksize=chunksize,
    alpha="auto",
    eta="auto",
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every
)

In [90]:
#Sort to arrange topics based on priority or similarity. 
# def Sort(sub_li):
#     sub_li.sort(key = lambda x: x[1])
#     sub_li.reverse()
#     return(sub_li)

# for story_num in range(len(stories)): 
#     test_doc = corpus[story_num]
#     vector = lda_model[test_doc]
#     new_vector = Sort(vector)
#     logfile = open('Stories/Topic to Story Relation.txt', 'w')
#     print(f"Topic_{story_num} is most similar to {new_vector}", file = logfile)   #last story is most similar with topic
#     logfile.close()
    
for i in range(10):  
    Lda_topics = open('Stories/LDA Model Topics.txt', 'a')
    print(new_model.show_topics(num_topics=10, num_words=10, formatted=True)[i],"\n", file = Lda_topics)
Lda_topics.close()

In [67]:
lda_model.print_topics()

[(0,
  '0.025*"work" + 0.017*"people" + 0.014*"school" + 0.011*"friend" + 0.011*"take" + 0.011*"think" + 0.011*"good" + 0.008*"one" + 0.008*"undergrad" + 0.008*"free"'),
 (1,
  '0.022*"good" + 0.015*"family" + 0.015*"lot" + 0.015*"grow" + 0.012*"poor" + 0.012*"year" + 0.012*"time" + 0.012*"get" + 0.010*"kid" + 0.010*"move"'),
 (2,
  '0.015*"work" + 0.015*"just" + 0.015*"know" + 0.012*"school" + 0.012*"take" + 0.012*"well" + 0.012*"parent" + 0.012*"people" + 0.009*"hard" + 0.009*"afford"'),
 (3,
  '0.001*"respect" + 0.001*"rent" + 0.001*"retirement" + 0.001*"retire" + 0.001*"rest" + 0.001*"schedule" + 0.001*"requisite" + 0.001*"refuse" + 0.001*"relentless" + 0.001*"sea"'),
 (4,
  '0.029*"get" + 0.022*"just" + 0.016*"class" + 0.010*"go" + 0.010*"never" + 0.010*"student" + 0.010*"work" + 0.010*"so" + 0.010*"food" + 0.009*"money"'),
 (5,
  '0.026*"job" + 0.020*"community" + 0.020*"well" + 0.020*"get" + 0.014*"college" + 0.014*"start" + 0.014*"able" + 0.014*"experience" + 0.014*"tuition" + 

In [ ]:
#lda_model.save("Models/test_model_1.model")

### Top topics detected by model (visualize for clusters)

In [ ]:
# top_topics = lda_model.top_topics(corpus)

# # Average topic coherence is the sum of topic coherences of all topics, divided by the number of topics.
# avg_topic_coherence = sum([t[1] for t in top_topics]) / num_topics
# print(f'Average topic coherence: %.4f. % avg_topic_coherence')

# from pprint import pprint
# print(top_topics)

### Saving/loading the LDA Model (just experimental)

In [39]:
new_model = gensim.models.ldamodel.LdaModel.load("Models/test_model_1.model")

In [53]:
#testing the loaded model
test_doc = corpus[-1]

vector = new_model[test_doc]

# logfile = open('Stories/Story_1.txt', 'a')
# print(new_model.show_topics(num_topics=10, num_words=10, formatted=False), file = logfile)
# logfile.close()

#Sort to arrange topics based on priority or similarity. 
def Sort(sub_li):
    sub_li.sort(key = lambda x: x[1])
    sub_li.reverse()
    return(sub_li)
new_vector = Sort(vector)
print(new_vector)#last story is most similar with topic 2

[(2, 0.33460167), (4, 0.2593409), (9, 0.15352553), (1, 0.10114299), (6, 0.090423174), (5, 0.059933234)]


# Visualization

In [46]:
#use "pip install pyLDAvis" if the pyLDAvis library isn't installed on your system
import pyLDAvis
import pyLDAvis.gensim_models

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(new_model, corpus, id2word, mds="mmds", R=30)
vis

C:\Users\MoRevolution\AppData\Roaming\Python\Python310\site-packages\pyLDAvis\_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.028258  0.177256       1        1  30.395113
4     -0.034860 -0.188031       2        1  24.372571
9     -0.178037 -0.062258       3        1  21.059501
5     -0.136492  0.068363       4        1   7.922141
6      0.094255 -0.113798       5        1   6.097716
1      0.156987  0.014087       6        1   5.242637
3      0.084590  0.105720       7        1   4.874612
7      0.013928 -0.000447       8        1   0.012513
8      0.013941 -0.000446       9        1   0.011857
0      0.013947 -0.000446      10        1   0.011338, topic_info=             Term       Freq      Total Category  logprob  loglift
57            job  18.000000  18.000000  Default  30.0000  30.0000
124          work  22.000000  22.000000  Default  29.0000  29.0000
95         school  20.000000  20.000000  Default  28.0000  28.0000
41            get  16.000000  16.000000  Default  27.0000  27.0000
140          good  13.000000  13.000000  Default  26.0000  26.0000
..            ...        ...        ...      ...      ...      ...
25      different   0.000244   1.876442  Topic10  -6.7776   0.1368
26         dollar   0.000244   2.236843  Topic10  -6.7776  -0.0389
27           drop   0.000244   4.009487  Topic10  -6.7776  -0.6225
28           drug   0.000244   4.042358  Topic10  -6.7776  -0.6307
29   emancipation   0.000244   1.364452  Topic10  -6.7776   0.4554

[608 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
127       1  0.177766        able
127       2  0.177766        able
127       3  0.533297        able
815       6  0.956115  absolutely
0         2  0.732895       abuse
...     ...       ...         ...
126       2  0.256063        year
126       3  0.192048        year
126       4  0.128032        year
466       7  0.995994       yield
620       2  0.894174       young

[651 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 5, 10, 6, 7, 2, 4, 8, 9, 1])

In [ ]:
with 
logfile = open('/yourfile.txt', 'a')
print>>logfile, lda.show_topics(topics=-1, topn=10)